<a href="https://colab.research.google.com/github/Existanze54/sirius-machine-learning-2025/blob/main/Seminars/BioInf/S8_Tree.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Основные методы ML

### Семинар 8: Деревья решений и случайный лес

Максимальная глубина, связь с variance. Bagging деревьев. One-hot кодирование. Оптимизация параметров GridSearchCV

In [ ]:
import scipy
import numpy as np
import pandas as pd

import itertools
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.metrics import matthews_corrcoef

from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, BaggingClassifier
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.model_selection import KFold, GridSearchCV, train_test_split

### Задача 1: Bias-variance trade-off


Продемонстрируйте bias-variance trade-off на примере DecisionTreeRegressor на представленном датасете, используя функцию plot_regression_predictions и приведенный ниже код (можете модифицировать).

*Подсказка*: попробуйте несколько значений глубины *max_depth* в пределах от 2 до 12 и сравните с дефолтным None. Также можете попробовать варьировать другие параметры.

In [ ]:
np.random.seed(42)
m = 200
X = np.random.rand(m, 1) * 3 - 1
y = 4 * (X - 0.5) ** 3
y = y + np.random.randn(m, 1) / 10

# визуализируем
plt.scatter(X,y,s=10)
plt.xlabel('x')
plt.ylabel('y')
plt.show()

In [ ]:
def plot_regression_predictions(tree_reg, X, y, ax, limits=[0, 1, -1, 1], title=""):
    """Передайте обученное дерево, координаты кривой Х и y,
        объект ax для отрисовки, опционально - координаты границ
         и заголовок графика"""
    x1 = np.linspace(limits[0], limits[1], 500).reshape(-1, 1)
    y_pred = tree_reg.predict(x1)
    ax.axis(limits)
    ax.set_title(title)
    ax.plot(X, y, "b.")
    ax.plot(x1, y_pred, "r.-", alpha=0.1, linewidth=1, label="y")

In [ ]:
# Замените "?" рабочим кодом либо напишите полностью свое решение
fig, axes = plt.subplots(2,3, figsize=(12,15))
axes = axes.flat

for i, d in enumerate( ?? ):
    axes[i].set(xlabel="$x_1$", ylabel="$y$")
    axes[i].label_outer()
    model = DecisionTreeRegressor(max_depth=??).fit(??, ?)
    plot_regression_predictions(???, ??, ?, axes[i], limits=[0, 1, -1, 1], title=f"Max depth: {d}")

In [ ]:
# Выводы - ?

### Задача 2: Качество Bagging с различными базовыми моделями

Попробуем оценить эффект бэггинга на качестве различных моделей классификации: логистической регрессии, SVC и дерева решений.

На датасете *breast_cancer* обучите модели Decision Tree, SVC (с параметрами по умолчанию) и логистическую регрессию.

Полученные предсказания на тесте оцените при помощи корреляции мэтьюса (**sklearn.metrics.matthews_corrcoef**, почитать про пользу этой метрики можно [здесь](https://bmcgenomics.biomedcentral.com/articles/10.1186/s12864-019-6413-7)), построив 90% доверительные интервалы при помощи реализующей bootstrap функции *bootstrap_metric*. Результат отобразите в виде боксплотов.

Обучите **BaggingClassifier** (предварительно почитав [документацию](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.BaggingClassifier.html)) с теми же самыми моделями в качестве базового эстиматора и числом эстиматоров равным 100.

На новом графике отобразите боксплоты для всех 6 моделей. Сделайте выводы.

In [ ]:
from sklearn.datasets import load_breast_cancer

def bootstrap_metric(x, y,
                    metric_fn,
                    samples_cnt=1000,
                    random_state=777):
    size = len(x)
    np.random.seed(random_state)
    b_metric = np.zeros(samples_cnt)
    for it in range(samples_cnt):
        poses = np.random.choice(x.shape[0], size=x.shape[0], replace=True)

        x_boot = x[poses]
        y_boot = y[poses]

        m_val = metric_fn(x_boot, y_boot)
        b_metric[it] = m_val

    return b_metric

In [ ]:
breast_cancer = load_breast_cancer()
#print(breast_cancer.DESCR)

In [ ]:
X = breast_cancer.data
y = breast_cancer.target
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=777)

Обучите базовые модели и сравните качество:

In [ ]:
logr_model = LogisticRegression(solver='liblinear',\
                                max_iter=1000).fit(X_train, y_train)

dt_model = DecisionTreeClassifier().fit(??)

svc_model = SVC().fit(??)

In [ ]:
dt_pred    =   dt_model.predict(X_test)
svc_pred   =   # your code here
logr_pred  =   # your code here

In [ ]:
boot_score_dt = bootstrap_metric(y_test,
                                   dt_pred,
                                   metric_fn=lambda x, y: matthews_corrcoef(y_true=x,
                                                                                  y_pred=y))
boot_score_svc = bootstrap_metric(y_test,
                                   svc_pred,
                                   metric_fn=lambda x, y: matthews_corrcoef(y_true=x,
                                                                                  y_pred=y))
boot_score_logr = bootstrap_metric(y_test,
                                   logr_pred,
                                   metric_fn=lambda x, y: matthews_corrcoef(y_true=x,
                                                                                  y_pred=y))

In [ ]:
plt.figure(figsize=(12,8))
sns.boxplot(y=np.concatenate([boot_score_dt,
                              boot_score_svc,
                              boot_score_logr]),
             x=["DT"] * 1000 + ['SVC'] * 1000 + ['Log-reg'] * 1000 )
plt.show();

Создайте ансамбли BaggingClassifier с рассмотренными выше моделями в качестве базовых эстиматоров и числом эстиматоров равным 100. Получите для каждого из них предсказание и bootstrap-оценку. Добавьте полученные данные на график.

In [ ]:
bagged_svc_model = BaggingClassifier(SVC(), \
                                     n_estimators=100).fit(X_train, y_train)
# your code here
# your code here

In [ ]:
b_dt_pred    =  bagged_dt_model.predict(X_test)
# your code here
# your code here

In [ ]:
matthews_m = lambda x, y: matthews_corrcoef(y_true=x, y_pred=y)

# your code here

In [ ]:
plt.figure(figsize=(12,8))
sns.boxplot(y=np.concatenate([boot_score_dt,
                              b_boot_score_dt,
                              boot_score_svc,
                              b_boot_score_svc,
                              boot_score_logr,
                              b_boot_score_logr]),
             x=["DT"] * 1000 + ["Bagged DT"] * 1000 + ['SVC'] * 1000 \
             + ['Bagged SVC'] * 1000 + ['Log-reg'] * 1000 + ['Bagged Log-reg'] * 1000)
plt.show()

In [ ]:
# Вывод -?

### Задача 3: Древесные модели и работа с признаками.

Поработаем с  [датасетом](https://www.kaggle.com/ronitf/heart-disease-uci), содержащим признаковые описания здоровых людей и пациентов с заболеваниями сердца.

In [ ]:
!wget https://data.bioml.ru/htdocs/courses/python/datasci/pandas/data/heart_uci_cleveland.csv -O heart.csv

In [ ]:
heart_dataset = pd.read_csv("heart.csv")
print(heart_dataset.shape)
print(heart_dataset.columns.values)
X = heart_dataset.drop("condition", axis=1)
y = heart_dataset['condition'] > 0
X_train, X_test, y_train, y_test = train_test_split(X, y.values, random_state=777)

In [ ]:
heart_dataset

#### Эксперимент: обучение классификаторов на вещественных признаках

Качество алгоритмов может зависеть не только от значений гиперпараметров, но и от предобратки исходных признаков. Некоторые из рассмотренных нами алгоритмов чувствительны к масштабу признаков. Посмотрим, насколько различны распределения признаков. Постройте гистограммы для признаков 'age', 'trestbps', 'chol', 'thalach', 'oldpeak'.

Глядя на получившиеся графики, скажите в чем заключается особенность данных? На какие алгоритмы это может повлиять? Почему?

In [ ]:
fig, axes = plt.subplots(3,2, figsize=(12,10))
axes = axes.flat
for i, name in enumerate( ??? ):
    heart_dataset[name].hist(ax=axes[i])
    axes[i].set_title(name)
fig.tight_layout()

Масштабирование признаков можно выполнить, например, одним из следующих способов:

$x_{new} = \dfrac{x - \mu}{\sigma}$, где $\mu, \sigma$ — среднее и стандартное отклонение значения признака по всей выборке;

$x_{new} = \dfrac{x - x_{min}}{x_{max} - x_{min}}$, где $[x_{min}, x_{max}]$ —  интервал наблюдаемых значений признака.

Такие схемы масштабирования приведены в классах [StandardScaler](http://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html#sklearn.preprocessing.StandardScaler) и [MinMaxScaler](http://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.MinMaxScaler.html#sklearn.preprocessing.MinMaxScaler).

Отберите все вещественные признаки и масштабируйте их одним из указанных способов.




In [ ]:
from sklearn.preprocessing import ??Scaler


В этой задаче будем рассматривать алгоритм *RandomForestClassifier*.

Для начала оцените среднее и std качества работы алгоритма с параметрами по умолчанию на масштабированных и исходных вещественных признаках из тренировочной выборки при помощи функции [cross_val_score](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_val_score.html). По умолчанию она оценивает качество работы модели по 5 фолдам.


Сделайте вывод о необходимости масштабировать признаки перед подачей их в древесные модели.

In [ ]:
from sklearn.model_selection import cross_val_score

cross_val_score(estimator=RandomForestClassifier(), X=??, y=??)

In [ ]:
# Вывод

#### Добавление категориальных признаков в модель

Вспомним, что у нас еще есть категориальные признаки. Случайный лес умеет работать с ними напрямую в том виде, в котором они в датасете находятся сейчас, однако многие другие модели требуют дополнительной предобработки.

Преобразуйте все категориальные признаки с помощью метода one-hot-encoding (например, это можно сделать с помощью функции [pandas.get_dummies](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html)). Передайте в функцию список колонок, которые необходимо закодировать.

После кодирования признаков разбейте датасет заново на тренировочную и тестовую выборки тем же способом, что и раньше.

In [ ]:
heart_dataset_transformed = pd.get_dummies(heart_dataset, columns = [ ??? ])

Обучим случайный лес.

Подберите для него оптимальные значения гиперпараметров, а именно параметр глубины (*max_depth*), критерий разбиения в деревьях (*criterion*) и максимальное число признаков *max_features*.

Для подбора гиперпараметров воспользуйтесь перебором по сетке, который реализован в классе [GridSearchCV](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html). Для ускорения вычислений можете передать параметр *n_jobs = -1*.

Обратите внимание, что эта операция может быть ресурсо- и трудоемкой! Не используйте большие массивы перебираемых значений параметров.

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier

In [ ]:
params = {
    "n_estimators": [1, 10, 100],
    "max_depth": [2, 5, 7, 10, 15, None],
    "criterion": ["gini", "entropy"],
    "max_features": ["sqrt", "log2", None]
}

grid = GridSearchCV(???, params, verbose=3, n_jobs=-1)
grid.fit(??, ?)

Оцените качество предсказания обученной на всех признаках модели на тесте. Используйте classification_report.

In [ ]:
# Выводы